# 1. Let's import necessary libraries

In [ ]:
import pyemu
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import matplotlib.pyplot as plt
import psutil
import shutil
import numpy as np
import sys
import flopy

## work with the latest version of swatmf

In [ ]:
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(1, path)
from swatmf import swatmf_pst_utils

# 2. Set up and write swatmf.con file

In [ ]:
swatmf_pst_utils.create_swatmf_con?

In [ ]:
# working directory
swatmf_model = "D:/test/middle_bosque_1000/SWAT-MODFLOW"
swat_model = "D:/test/middle_bosque_1000/SWAT"

In [ ]:
# calibration period
sim_start = '1/1/1985'
warmup = 0
cal_start = '1/1/1985'
cal_end = '12/31/1985'

# time step
time_step = 'day'

# locations (what our targets)
subs = [58]
grids = [501]

In [ ]:
swatmf_pst_utils.create_swatmf_con(
    swatmf_model, sim_start, warmup, cal_start, cal_end, subs=subs, grids=grids)

## 2.1 PEST initial setup

In [ ]:
# copy all necessary files (exes) to your working direcotry
swatmf_pst_utils.init_setup(swatmf_model, swat_model)

In [ ]:
# check MODFLOW model
mname = "modflow.mfn"
m = flopy.modflow.Modflow.load(mname,model_ws=swatmf_model)
m.check()

# 3. Build template files

In [ ]:
os.chdir(swatmf_model)

## 3.1 MODFLOW pval

In [ ]:
# pval file
pval_file = 'mf_1000.pval'

In [ ]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
gw_par

## 3.2 SWAT model.in file

In [ ]:
# model.in file used
sw_par = swatmf_pst_utils.model_in_to_template_file()
sw_par

# 4. Build instruction files

### Let's do initial run!

## 4.1 Streamflow (SWAT)

In [ ]:
# extract daily stream discharge
swatmf_pst_utils.extract_day_stf(subs, sim_start, warmup, cal_start, cal_end)

## 4.2 match it with stf_obd file (SWAT)

In [ ]:
swatmf_pst_utils.stf_obd_to_ins('stf_058.txt', 'rch058',cal_start, cal_end)

## 4.3 Depth to watertable (MODFLOW) 

In [ ]:
mf_obs_grid_ids = pd.read_csv(
                    os.path.join(swatmf_model, 'modflow.obs'),
                    sep=r'\s+',
                    usecols=[3, 4],
                    skiprows=2,
                    header=None
                    )

In [ ]:
mf_sim = pd.read_csv(
                     os.path.join(swatmf_model, 'swatmf_out_MF_obs'), skiprows=1, sep=r'\s+',
                    names=col_names,
                    # usecols=grids,
                    )
mf_sim.index = pd.date_range(sim_start, periods=len(mf_sim))
mf_sim = mf_sim[sim_start:cal_end]

In [ ]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)

## 4.4 match it with modflow.obd file (MODFLOW)

In [ ]:
swatmf_pst_utils.mf_obd_to_ins('dtw_501.txt', 'g_5699', cal_start, cal_end)

# 5. Create PEST control file

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'mb_zon_dummy.pst')

In [ ]:
par = pst.parameter_data
par

## 5.1 Assign parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

## 5.2 Adjust initial parameter values and their ranges

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1


## 5.3 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 5.4 Provide actual observed values to control file

In [ ]:
os.getcwd()

In [ ]:
# Streamflow
stf_obd = pd.read_csv('swat_rch_day.obd',
                       sep=r'\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# watertable
dtw_obd = pd.read_csv('dtw_day.obd',
                       sep=r'\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 6. Create new control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [ ]:
pst.control_data.formatted_values

In [ ]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

### add new PEST++ variables like so:

In [ ]:
# A few examples of adding PEST++ options of different types:
# pestpp-ies; the number of realizations to draw in order to form parameter and observation ensembles.
pst.pestpp_options['ies_num_reals'] = 50

# specifies a list of values for the Marquardt lambda used in calculation of parameter upgrades. 
# pst.pestpp_options["lambdas"] = [0.1, 1, 10, 100, 1000]
# pst.pestpp_options["ies_bad_phi_sigma"] = 2.0

# pestpp-da; True/False, specify whether to use the simulated states at the end of each cycle as the initial states for the next cycle.   
# pst.pestpp_options['da_use_simulated_states'] = True

# check the dictionary again
pst.pestpp_options

In [ ]:
pst.write('mb_zon.pst', version=2)

You can also read and load the existing pest control file.

In [ ]:
pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
pst_read.parameter_data